In [ ]:
#|hide
from fastrl.test_utils import initialize_notebook
initialize_notebook()

In [ ]:
#|default_exp pipes.iter.firstlast

In [ ]:
#|export
# Python native modules
import warnings
# Third party libs
from fastcore.all import add_docs
import torchdata.datapipes as dp

import torch
# Local modules
from fastrl.core import StepType

# FirstLast
> DataPipe for merging multiple `StepTypes` into a single one by keeping the first and last steps.

In [ ]:
#|export
class FirstLastMerger(dp.iter.IterDataPipe):
    def __init__(self, 
                 source_datapipe, 
                 gamma:float=0.99
        ):
        self.source_datapipe = source_datapipe
        self.gamma = gamma
        
    def __iter__(self) -> StepType:
        self.env_buffer = {}
        for steps in self.source_datapipe:
            if not isinstance(steps,(list,tuple)):
                raise ValueError(f'Expected {self.source_datapipe} to return a list/tuple of steps, however got {type(steps)}')
                
            if len(steps)==1:
                yield steps[0]
                continue
                
            fstep,lstep = steps[0],steps[-1]
            
            reward = fstep.reward
            for step in steps[1:]:
                reward*=self.gamma
                reward+=step.reward
                
            yield fstep.__class__(
                state=torch.tensor(fstep.state),
                next_state=torch.tensor(lstep.next_state),
                action=fstep.action,
                terminated=lstep.terminated,
                truncated=lstep.truncated,
                reward=reward,
                total_reward=lstep.total_reward,
                env_id=lstep.env_id,
                proc_id=lstep.proc_id,
                step_n=lstep.step_n,
                episode_n=fstep.episode_n,
                image=fstep.image
            )
                
add_docs(
    FirstLastMerger,
    """Takes multiple steps and converts them into a single step consisting of properties
    from the first and last steps. Reward is recalculated to factor in the multiple steps.""",
)

In [ ]:
#|hide
# Used here to avoid UserWarnings related to gym complaining about bounding box / action space format.
# There must be a bug in the CartPole-v1 env that is causing this to show. Also couldnt figure out the 
# regex, so instead we filter on the lineno, which is line 98.
warnings.filterwarnings("ignore",category=UserWarning,lineno=98)

Below we see an example where we collect 2 steps for each env, **then** yield them. This is useful for
training models of larger chunks of env step output.

In [ ]:
import pandas as pd
import gymnasium as gym
from fastcore.all import test_eq
from fastrl.core import test_len
from fastrl.envs.gym import GymStepper
from fastrl.pipes.iter.nstep import NStepper

In [ ]:
def first_last_test(envs,total_steps,n=1,seed=0):
    pipe = dp.map.Mapper(envs)
    pipe = pipe.map(gym.make)
    pipe = dp.iter.MapToIterConverter(pipe)
    pipe = dp.iter.InMemoryCacheHolder(pipe)
    pipe = pipe.cycle()
    pipe = GymStepper(pipe,seed=seed)
    pipe = NStepper(pipe,n=n)
    pipe = FirstLastMerger(pipe)
    pipe = pipe.header(total_steps)
    return list(pipe)

steps = first_last_test(['CartPole-v1']*3,200,2,0)
pd.DataFrame(steps)[['state','next_state','env_id','terminated']][:10]

/usr/local/lib/python3.8/dist-packages/torchdata/datapipes/iter/util/header.py:57: UserWarning: The length of this HeaderIterDataPipe is inferred to be equal to its limit.The actual value may be smaller if the actual length of source_datapipe is smaller than the limit.
  warn(
/home/fastrl_user/fastrl/fastrl/envs/gym.py:122: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(step.next_state),
/tmp/ipykernel_12097/1503746987.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(fstep.state),
/tmp/ipykernel_12097/1503746987.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().deta

,state,next_state,env_id,terminated
0,"[tensor(0.0137), tensor(-0.0230), tensor(-0.0459), tensor(-0.0483)]","[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]",tensor(139964952847600),tensor(False)
1,"[tensor(0.0137), tensor(-0.0230), tensor(-0.0459), tensor(-0.0483)]","[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]",tensor(139964953074224),tensor(False)
2,"[tensor(0.0137), tensor(-0.0230), tensor(-0.0459), tensor(-0.0483)]","[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]",tensor(139964952848080),tensor(False)
3,"[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]","[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]",tensor(139964952847600),tensor(False)
4,"[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]","[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]",tensor(139964953074224),tensor(False)
5,"[tensor(0.0132), tensor(0.1727), tensor(-0.0469), tensor(-0.3552)]","[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]",tensor(139964952848080),tensor(False)
6,"[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]","[tensor(0.0353), tensor(0.3702), tensor(-0.0866), tensor(-0.7006)]",tensor(139964952847600),tensor(False)
7,"[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]","[tensor(0.0353), tensor(0.3702), tensor(-0.0866), tensor(-0.7006)]",tensor(139964953074224),tensor(False)
8,"[tensor(0.0167), tensor(0.3685), tensor(-0.0540), tensor(-0.6622)]","[tensor(0.0353), tensor(0.3702), tensor(-0.0866), tensor(-0.7006)]",tensor(139964952848080),tensor(False)
9,"[tensor(0.0241), tensor(0.5643), tensor(-0.0672), tensor(-0.9714)]","[tensor(0.0427), tensor(0.1763), tensor(-0.1007), tensor(-0.4364)]",tensor(139964952847600),tensor(False)


First, `NStepper(pipe,n=1)` with `FirstLastMerger` should be identical to a pipelines that never used it.

In [ ]:
pipe = dp.map.Mapper(['CartPole-v1']*3)
pipe = pipe.map(gym.make)
pipe = dp.iter.MapToIterConverter(pipe)
pipe = dp.iter.InMemoryCacheHolder(pipe)
pipe = pipe.cycle()
pipe = GymStepper(pipe,seed=0)
pipe = pipe.header(10)

no_n_steps = list(pipe)
steps = first_last_test(['CartPole-v1']*3,10,1,0)

/usr/local/lib/python3.8/dist-packages/torchdata/datapipes/iter/util/header.py:57: UserWarning: The length of this HeaderIterDataPipe is inferred to be equal to its limit.The actual value may be smaller if the actual length of source_datapipe is smaller than the limit.
  warn(
/home/fastrl_user/fastrl/fastrl/envs/gym.py:122: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(step.next_state),


If `n=1` we should expect that regardless of the number of envs, both n-step and simple environment
pipelines should be identical.

In [ ]:
test_len(steps,no_n_steps)
for field in ['next_state','state','terminated']:
    for i,(step,no_n_step) in enumerate(zip(steps,no_n_steps)): 
        test_eq(getattr(step,field),getattr(no_n_step,field))

In [ ]:
#|export
def n_first_last_steps_expected(
    default_steps:int, # The number of steps the episode would run without n_steps
):
    return default_steps 
    
n_first_last_steps_expected.__doc__=r"""
This function doesnt do much for now. `FirstLastMerger` pretty much undoes the number of steps `nsteps` does.
"""    

In [ ]:
expected_n_steps = n_first_last_steps_expected(default_steps=18)
print('Given the above values, we expect a single episode to be ',expected_n_steps,' steps long')
steps = first_last_test(['CartPole-v1']*1,expected_n_steps+1,2,0)
# The first episode should have ended on row 34, beign 35 steps long. The 36th row should be a new episode
test_eq(steps[-2].terminated,torch.tensor([True]))
test_eq(steps[-2].episode_n,torch.tensor([1]))
test_eq(steps[-2].step_n,torch.tensor([18]))
test_eq(steps[-1].terminated,torch.tensor([False]))
test_eq(steps[-1].episode_n,torch.tensor([2]))
test_eq(steps[-1].step_n,torch.tensor([2])) # Main difference, the "step" for the new episode will be 2 instead of 1

Given the above values, we expect a single episode to be  18  steps long


/usr/local/lib/python3.8/dist-packages/torchdata/datapipes/iter/util/header.py:57: UserWarning: The length of this HeaderIterDataPipe is inferred to be equal to its limit.The actual value may be smaller if the actual length of source_datapipe is smaller than the limit.
  warn(
/home/fastrl_user/fastrl/fastrl/envs/gym.py:122: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(step.next_state),
/tmp/ipykernel_12097/1503746987.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(fstep.state),
/tmp/ipykernel_12097/1503746987.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().deta

In [ ]:
expected_n_steps = n_first_last_steps_expected(default_steps=18)
print('Given the above values, we expect a single episode to be ',expected_n_steps,' steps long')
steps = first_last_test(['CartPole-v1']*1,expected_n_steps+1,4,0)
# The first episode should have ended on row 34, beign 35 steps long. The 36th row should be a new episode
test_eq(steps[-2].terminated,torch.tensor([True]))
test_eq(steps[-2].episode_n,torch.tensor([1]))
test_eq(steps[-2].step_n,torch.tensor([18]))
test_eq(steps[-1].terminated,torch.tensor([False]))
test_eq(steps[-1].episode_n,torch.tensor([2]))
test_eq(steps[-1].step_n,torch.tensor([4]))

Given the above values, we expect a single episode to be  18  steps long


/tmp/ipykernel_12097/1503746987.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(fstep.state),
/tmp/ipykernel_12097/1503746987.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  next_state=torch.tensor(lstep.next_state),


In [ ]:
expected_n_steps = n_first_last_steps_expected(default_steps=18)
print('Given the above values, we expect a single episode to be ',expected_n_steps,' steps long')
steps = first_last_test(['CartPole-v1']*3,expected_n_steps*3+1,2,0)
# The first episode should have ended on row 34, beign 35 steps long. The 36th row should be a new episode
test_eq(steps[-2].terminated,torch.tensor([True]))
test_eq(steps[-2].episode_n,torch.tensor([1]))
test_eq(steps[-2].step_n,torch.tensor([18]))
test_eq(steps[-1].terminated,torch.tensor([False]))
test_eq(steps[-1].episode_n,torch.tensor([2]))
test_eq(steps[-1].step_n,torch.tensor([2]))

Given the above values, we expect a single episode to be  18  steps long


/tmp/ipykernel_12097/1503746987.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(fstep.state),
/tmp/ipykernel_12097/1503746987.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  next_state=torch.tensor(lstep.next_state),


In [ ]:
#|hide
#|eval: false
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev import nbdev_export
    nbdev_export()

/usr/local/lib/python3.8/dist-packages/nbdev/export.py:54: UserWarning: Notebook '/home/fastrl_user/fastrl/nbs/07_Agents/02_Continuous/12u_agents.ppo.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
